## Combine 2011 and 2017 census data through fuzzy name matching

This notebook uses fuzzy string matching to identify overlaps in two census datasets. Matches are defined as being 95% similar (Levenshtein distance) and requiring that the person never moves backwards through the progression of tenure-track titles (i.e. a "Full Professor" never becomes an "Assistant Professor")

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import uuid

In [2]:
thresholds = [90, 95, 100]

# Load 2011 data
df2011 = pd.read_csv('./files/2011_faculty.tsv', sep='\t', header=None)
df2011.columns = ['name', 'abbrv_name', 'title', 'place', 'year', 'sex']

# Load 2017 data
df2017 = pd.read_csv('./files/2017_faculty.tsv', sep='\t', header=None)
df2017.columns = ['name', 'abbrv_name', 'title', 'place', 'year']

"""
One possibility would be to match based on name directly:
df2011.merge(df2017, on='abbrv_name', suffixes=("_2011", "_2017"))

Suppose we want some flexibility in the matching, though. That's below.
"""
pass

In [3]:
def get_mapping(threshold=90):
    """ Find a mapping to align 2011 and 2017 names, 
    within a specified threshold (percent similarity). """
    mapping = []
    names = choices=df2017['name'].values

    for i, row in df2011.iterrows():
        matches = process.extract(row['name'], names, limit=25)
        for match in matches:
            if match[1] >= threshold:
                matching_rows = df2017.query('name == "%s"' % match[0])
                for j, _ in matching_rows.iterrows():
                    mapping.append((i, j, match[1]))
    return mapping

# Get mappings for minimum threshold
mapping = get_mapping(95)

## Unpack matches, merge into tables, save to CSVs

In [153]:
# Define typical ordering of titles
titles = {"Assistant Professor":0, "Associate Professor":1, "Full Professor":2}

In [5]:
maps = {}

# For each mapping level, get the set of best matches.
for threshold in [95]:
    temp_map = {}
    for i, j, ratio in mapping:
        
        title_i = titles.get(df2011.iloc[i]['title'], -1)
        title_j = titles.get(df2017.iloc[j]['title'], -1)
        
        # 1) Fuzzy matching ratio should be larger than the threshold
        # 2) There isn't a better match available
        # 3) The 2017 title doesn't regress from 2011
        if ratio >= threshold and i not in temp_map and title_i <= title_j:
            temp_map[i] = j
    maps[threshold] = temp_map

In [8]:
df2017['match_col'] = np.arange(len(df2017))

# Make that set of matches a column (so we can join the two tables)
threshold = 95
link_ids = [maps[threshold].get(i, -1) for i in range(len(df2011))]
df2011['match_col'] = link_ids
merged = df2011.merge(df2017, how='outer', on='match_col', suffixes=("_2011", "_2017"))

In [31]:
a = np.sum(merged['name_2011'].isnull())
b = np.sum(merged['name_2017'].isnull())
c = len(merged)

print(a)
print(b)
print(c - a - b)
print(len(merged))

1776
1556
3461
6793


In [22]:
cleaned = merged[['title_2011', 'place_2011', 'title_2017', 'place_2017', 'sex']].copy()
uids = [uuid.uuid4().hex for i,_ in merged.iterrows()]
cleaned['guid'] = uids

def standardize_institution_name(institution):
    if institution == 'State University of New York, Albany':
        institution = "University of Albany, SUNY"
    elif institution == 'UC Santa Barbara':
        institution = "University of California, Santa Barbara"
    elif institution == 'State University of New York, Binghamton':
        institution = "Binghamton University, SUNY"
    elif institution == 'State University of New York, Stony Brook':
        institution = "Stony Brook University, SUNY"
    elif institution == 'MIT':
        institution = "Massachusetts Institute of Technology"
    elif institution == 'UC Davis':
        institution = "University of California, Davis"
    elif institution == 'State University of New York, Buffalo':
        institution = "University of Buffalo, SUNY"
    elif institution == 'UC Irvine':
        institution = "University of California, Irvine"
    elif institution == "UC Riverside":
        institution = "University of California, Riverside"
    elif institution == 'Missouri University of Science and Technology':
        institution = "Missouri University of Science & Technology"
    elif institution == "University of Kansas, Lawrence":
        institution = "University of Kansas"
    elif institution == "Oregon Health and Science University":
        institution = "Oregon Health & Science University"
    elif institution == "UCLA":
        institution = "University of California, Los Angeles"
    elif institution == "UC Berkeley":
        institution = "University of California, Berkeley"
    elif institution == "Toyota Technological Institute at Chicago":
        institution = "Toyota Technological Institute, Chicago"
    elif institution == "College of William and Mary":
        institution = "College of William & Mary"
    elif institution == "Oakland University (Michigan)":
        institution = "Oakland University"
    elif institution == "University of Minnesota, Minneapolis":
        institution = "University of Minnesota"
    elif institution == "Georgia Tech":
        institution = "Georgia Institute of Technology"
    elif institution == "Queens University":
        institution = "Queen's University"
    elif institution == "Washington State University, Pullman":
        institution = "Washington State University"
    elif institution == "University of North Texas, Denton":
        institution = "University of North Texas"
    elif institution == "UC San Diego":
        institution = "University of California, San Diego"
    elif institution == "Texas A&M":
        institution = "Texas A&M University"
    elif institution == "University of Hawaii, Manoa":
        institution = "University of Hawaii"
    elif institution == "University of Maryland, College Park":
        institution = "University of Maryland"
    elif institution == "Polytechnic Institute of NYU":
        institution = "Polytechnic Institute of New York University"
    elif institution == "UC Santa Cruz":
        institution = "University of California, Santa Cruz"
    elif institution == "CUNY Graduate Center":
        institution = "City University of New York, Graduate Center"
    elif institution == "Concordia University, Montreal":
        institution = "Concordia University"
    elif institution == "University of Idaho, Moscow":
        institution = "University of Idaho"
    elif institution == "New Mexico Institute of Mining and Technology":
        institution = "New Mexico Tech"
    elif institution == "Claremont Graduate University":
        institution = "Claremont Graduate University"

    return institution

places_2011 = []
for i, row in cleaned.iterrows():
    places_2011.append(standardize_institution_name(row['place_2011']))
    
cleaned['place_2011'] = places_2011
cleaned.to_csv('./cleaned_at_95.csv')
#merged.to_csv('./with_names.csv')
cleaned

,title_2011,place_2011,title_2017,place_2017,sex,guid
0,Assistant Professor,Dartmouth College,NaN,NaN,M,be49977642f740edbce2e5f55dbc0297
1,Associate Professor,University of Kansas,NaN,NaN,M,480f1efa35f94529926206fa58fe28a3
2,Associate Professor,Indiana University,NaN,NaN,M,118facd51d78441186cdd4f3041e3428
3,Full Professor,Massachusetts Institute of Technology,NaN,NaN,M,5b86808d7cb04543b5283ce24f4ff614
4,Associate Professor,Northwestern University,NaN,NaN,M,a276b3120d8545dbb4e771a4dab0a0b4
5,Associate Professor,Temple University,NaN,NaN,M,40fb6349c3694d3da6650aafdfa31c74
6,Full Professor,"University of Albany, SUNY",NaN,NaN,M,b43de9fbca3f49d2a1b17e6e5e4e6a90
7,Full Professor,Dalhousie University,NaN,NaN,M,4829a9a10e8b4b77a0a12010cdfdab3c
8,Full Professor,Northwestern University,NaN,NaN,M,bba3310cfd13408db620dface35c21c0
9,Full Professor,Carnegie Mellon University,NaN,NaN,M,c05f0ed20fc64c709c6f1a840006ca60


## Highest gain, attrition

### Change in department sizes

In [97]:
totals_2017 = cleaned.groupby('place_2017').count().reset_index()
totals_2017 = totals_2017.rename(index=str, columns={"place_2017":"place", "title_2017":"count"})
totals_2017 = totals_2017[['place', 'count']]

totals_2011 = cleaned.groupby('place_2011').count().reset_index()
totals_2011 = totals_2011.rename(index=str, columns={"place_2011": "place", "title_2011":"count"})
totals_2011 = totals_2011[['place', 'count']]

In [103]:
totals = totals_2017.merge(totals_2011, on='place', suffixes=['_2017', '_2011'])
totals

,place,count_2017,count_2011
0,Arizona State University,46,40
1,Auburn University,17,18
2,"Binghamton University, SUNY",25,19
3,Boston University,30,19
4,Brandeis University,13,12
5,Brigham Young University,31,28
6,Brown University,30,26
7,California Institute of Technology,9,16
8,Carleton University,27,28
9,Carnegie Mellon University,142,125


In [152]:
totals['perc_change'] = 100*(totals['count_2017'] - totals['count_2011']) / totals['count_2011']
totals = totals.sort_values(['perc_change']).reset_index()
totals.to_csv('./dept_totals.csv')
totals

,index,place,count_2017,count_2011,perc_change
0,33,Georgia Institute of Technology,31,89,-65.168539
1,77,Queen's University,18,35,-48.571429
2,7,California Institute of Technology,9,16,-43.750000
3,13,Claremont Graduate University,3,5,-40.000000
4,32,George Washington University,14,22,-36.363636
5,45,Massachusetts Institute of Technology,48,75,-36.000000
6,174,University of Southern Mississippi,11,17,-35.294118
7,69,Oregon Health & Science University,8,12,-33.333333
8,192,University of Wyoming,6,9,-33.333333
9,22,Dalhousie University,25,36,-30.555556


### Retention

In [150]:
retained = cleaned.query('place_2011 == place_2017').groupby('place_2011').count().reset_index()
retained = retained.rename(index=str, columns={"place_2011":"place", "title_2011":"count"})[['place', 'count']]

not_retained = cleaned.query('place_2011 != place_2017').groupby('place_2011').count().reset_index()
not_retained = not_retained.rename(index=str, columns={"place_2011":"place", "title_2011":"count"})[['place', 'count']]

retained = retained.merge(not_retained, on='place', suffixes=['_ret', '_not'])
retained['retained_frac'] = retained['count_ret'] / (retained['count_ret'] + retained['count_not'])
retained = retained.sort_values(['retained_frac'], ascending=False).reset_index()
retained.to_csv('./retention.csv')
retained

,index,place,count_ret,count_not,retained_frac
0,79,Rice University,17,1,0.944444
1,139,"University of Massachusetts, Boston",11,1,0.916667
2,151,"University of Nevada, Las Vegas",10,1,0.909091
3,3,Boston University,17,2,0.894737
4,82,Santa Clara University,8,1,0.888889
5,129,University of Iowa,15,2,0.882353
6,55,New Jersey Institute of Technology,22,3,0.880000
7,30,Florida State University,14,2,0.875000
8,167,University of Saskatchewan,21,3,0.875000
9,160,University of Oklahoma,13,2,0.866667
